In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [21]:
%run library.ipynb
%run utils.ipynb
%run math.ipynb

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to c:\users\jh\appdata\local\temp\pip-req-build-3jfh8v6y


  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' 'C:\Users\JH\AppData\Local\Temp\pip-req-build-3jfh8v6y'
'sudo'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'sudo'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'rm'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


ModuleNotFoundError: No module named 'matplotlib'

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
#GPU 사용
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [ ]:
title_data = pd.read_excel('/content/drive/MyDrive/news/title_test.xlsx')

In [ ]:
title_data.loc[(title_data['label'] == "a"), 'label'] = 0  
title_data.loc[(title_data['label'] == "b"), 'label'] = 1  
title_data.loc[(title_data['label'] == "c"), 'label'] = 2  

In [ ]:
title_list = []
for title, label in zip(title_data['title'], title_data['label'])  :
    data = []
    data.append(title)
    data.append(str(label))

    title_list.append(data)

In [ ]:
title_train, title_test = train_test_split(title_list, test_size=test_size, random_state=random_state)

In [5]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [6]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

train = BERTDataset(title_train, 0, 1, tok, max_len, True, False)
test = BERTDataset(title_test, 0, 1, tok, max_len, True, False)

using cached model


NameError: name 'title_train' is not defined

In [7]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=0)

NameError: name 'train' is not defined

In [8]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [9]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)


no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    


NameError: name 'learning_rate' is not defined

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [27]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)
    
    my_classifier.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval = []
        for i in out:
            
            logits=i
            logits = np.round(logits.detach().cpu().numpy(),2)
            
            for idx , j in enumerate(logits):
                logits[idx] = np.exp(j)
            tot_e = sum(logits)
            logits = (logits/tot_e) * 100
            #print(type(logits))
            if np.argmax(logits) == 0:
                test_eval.append("판단유보")
            elif np.argmax(logits) == 1:
                test_eval.append("책임회피")
            else : test_eval.append("선정주의")

        
       # print(">> 이타이틀은 판단유보 {:.2f}%, 책임회피 {:.2f}%, 선정주의 {:.2f}% 으로 측정되어 {}유형의 따옴표입니다".format(logits[0],logits[1],logits[2],test_eval[0]))
        return logits
        

using cached model
